In [ ]:
import sys
sys.path.append('../')
import tensorflow as tf
from src.models.FeedbackModelBuilder import VGG16FeedbackFrozen4BlockTo1Block, VGG16Frozen4BlockTo1Block
from src.DatasetNoise import tf_gaussian_noise, tf_salt_pepper_noise


batch_size = 8
num_classes = 16
img_size = 224
input_shape = (None, img_size, img_size, 3) 

path_to_datasets = '../../experiments/create_datasets/normalize_dataset/saved_datasets/'

test_ds_prepared_without_batch = tf.data.Dataset.load(os.path.join(path_to_datasets, "test_ds_without_batching"))

models = {
    "VGG16FeedbackFrozen4BlockTo1Block": VGG16FeedbackFrozen4BlockTo1Block, 
    "VGG16Frozen4BlockTo1Block": VGG16Frozen4BlockTo1Block
}
datasets = {"without_noise": test_ds_prepared_without_batch.batch(batch_size).prefetch(4),
           "gaussian_noise": test_ds_prepared_without_batch.map(tf_gaussian_noise).batch(batch_size).prefetch(4),
           "salt_pepper_noise": test_ds_prepared_without_batch.map(tf_salt_pepper_noise).batch(batch_size).prefetch(4)}

for model_name, model in models.items():
    for ds_name, test_ds in datasets.items():
        X_test = list(map(lambda x: x[0], test_ds))
        y_test = list(map(lambda x: x[1], test_ds))

        confusion_matrix = get_confusion_matrix_for_model_and_data(X_test, y_test)
        path_to_save = f"confusion_matr_{model_name}_on_{dataset_name}.csv"
        visualize_confusion_matrix(confusion_matrix, model_name = model_name, dataset_name = dataset_name, path_to_save = path_to_save)